In [ ]:
# https://qiita.com/DS27/items/1a79220f982e0094f1fc

# 欠損値の処理方法について整理してみた

In [8]:
# ライブラリーのインポート
import pandas as pd
import numpy as np
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import GradientBoostingRegressor

import warnings
warnings.simplefilter('ignore')

# データの読み込み（タイタニックデータ）
df = pd.read_csv('./train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# データの基本情報を確認
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
# 欠損値の確認
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
#性別の数値変換
df.loc[(df['Sex'] == 'male'), 'Sex'] = 0
df.loc[(df['Sex'] == 'female'), 'Sex'] = 1

# 不要なカラムの削除
df = df.drop(['Name',
              'Cabin',
              'Ticket',
              'SibSp',
              'Parch',
              'Embarked',
              'Fare'],
             axis=1)

df.head(10)

,PassengerId,Survived,Pclass,Sex,Age
0,1,0,3,0,22.0
1,2,1,1,1,38.0
2,3,1,3,1,26.0
3,4,1,1,1,35.0
4,5,0,3,0,35.0
5,6,0,3,0,NaN
6,7,0,1,0,54.0
7,8,0,3,0,2.0
8,9,1,3,1,27.0
9,10,1,2,1,14.0


## リストワイズ法

In [5]:
# 欠損値のサンプルを削除してしまう方法
df_listwise = df.dropna()
df_listwise.head(10)

,PassengerId,Survived,Pclass,Sex,Age
0,1,0,3,0,22.0
1,2,1,1,1,38.0
2,3,1,3,1,26.0
3,4,1,1,1,35.0
4,5,0,3,0,35.0
6,7,0,1,0,54.0
7,8,0,3,0,2.0
8,9,1,3,1,27.0
9,10,1,2,1,14.0
10,11,1,3,1,4.0


## 単一代入法

In [11]:
# 平均値補完
mean = df.mean()
df_mean = df.fillna(mean)
df_mean.head(10)

,PassengerId,Survived,Pclass,Sex,Age
0,1,0,3,0,22.000000
1,2,1,1,1,38.000000
2,3,1,3,1,26.000000
3,4,1,1,1,35.000000
4,5,0,3,0,35.000000
5,6,0,3,0,29.699118
6,7,0,1,0,54.000000
7,8,0,3,0,2.000000
8,9,1,3,1,27.000000
9,10,1,2,1,14.000000


In [12]:
# 中央値補完
median = df.median()
df_median = df.fillna(median)
df_median.head(10)

,PassengerId,Survived,Pclass,Sex,Age
0,1,0,3,0,22.0
1,2,1,1,1,38.0
2,3,1,3,1,26.0
3,4,1,1,1,35.0
4,5,0,3,0,35.0
5,6,0,3,0,28.0
6,7,0,1,0,54.0
7,8,0,3,0,2.0
8,9,1,3,1,27.0
9,10,1,2,1,14.0


In [13]:
# 回帰代入法（GBDT）
df_master = df.dropna()
df_miss = df[df.isnull().any(axis=1)]
model = GradientBoostingRegressor().fit(df_master.drop(["Age","Survived"], axis=1), df_master["Age"])
df_miss["Age"] = model.predict(df_miss.drop(["Age","Survived"], axis=1))

df_GBDT = pd.concat([df_master, df_miss]).sort_index()
df_GBDT.head(10)
# 6行目のAgeが30.534716で補完されている。
# これはAgeを目的変数とし、説明変数に「Passengerld, Pclass, Sex」を使用して
# GBDTで回帰モデルを構築した上で6行目のAgeを予測した結果で欠損値を補完している。

,PassengerId,Survived,Pclass,Sex,Age
0,1,0,3,0,22.000000
1,2,1,1,1,38.000000
2,3,1,3,1,26.000000
3,4,1,1,1,35.000000
4,5,0,3,0,35.000000
5,6,0,3,0,30.534716
6,7,0,1,0,54.000000
7,8,0,3,0,2.000000
8,9,1,3,1,27.000000
9,10,1,2,1,14.000000


In [16]:
# IterativeImputerによる代入法
from sklearn.impute import IterativeImputer

imp = IterativeImputer(max_iter=10, random_state=1)
df_IterativeImputer = pd.DataFrame(imp.fit_transform(df))
df_IterativeImputer.columns = df.columns
df_IterativeImputer.head(10)

,PassengerId,Survived,Pclass,Sex,Age
0,1.0,0.0,3.0,0.0,22.000000
1,2.0,1.0,1.0,1.0,38.000000
2,3.0,1.0,3.0,1.0,26.000000
3,4.0,1.0,1.0,1.0,35.000000
4,5.0,0.0,3.0,0.0,35.000000
5,6.0,0.0,3.0,0.0,26.181867
6,7.0,0.0,1.0,0.0,54.000000
7,8.0,0.0,3.0,0.0,2.000000
8,9.0,1.0,3.0,1.0,27.000000
9,10.0,1.0,2.0,1.0,14.000000


In [17]:
# KNNImputerによる代入法（K近傍法）
from missingpy import KNNImputer
imp = KNNImputer(n_neighbors=2, weights='uniform')
df_KNN = pd.DataFrame(imp.fit_transform(df))
df_KNN.columns = df.columns
df_KNN.head(10)

# missingpyない場合は、pipからインストールすれば良いと思ったが、エラーで出来ないかも。。
# pip install missingpy

ModuleNotFoundError: No module named 'sklearn.neighbors.base'

In [20]:
# MissForestによる代入法（ランダムフォレスト）
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from missingpy import MissForest
imp = MissForest(max_iter=10)
df_MF = pd.DataFrame(imp.fit_transform(df))
df_MF.columns = df.columns
df_MF.head(10)

ImportError: cannot import name '_check_weights' from 'sklearn.neighbors._base' (c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\neighbors\_base.py)